In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=3)

file = 'sensitivity_10_1e5.csv'

def mad(df):
    return (df - df.median()).abs().median()

In [2]:
sens = pd.DataFrame(np.loadtxt(f'../out/{file}'), columns=['r1', 'r2', 'r3', 't1', 't2', 't3', 'det', 'det_true'])\
            .filter(['r1', 'r2', 'r3', 'det', 'det_true'])
sens = sens.groupby(['r1', 'r2', 'r3']).agg([mad, np.median]).reset_index()
sens['ratio'] = sens[('det', 'mad')] / sens[('det', 'median')]

sens

r1    r2    r3           det             det_true            \
                               mad     median       mad    median   
0    0.1   0.1   0.1  3.427852e-04  11.488274  1.664354  4.308576   
1    0.1   0.1   1.0  2.445812e-03   9.317828  1.661183  4.302993   
2    0.1   0.1  10.0  5.822396e-04   6.845921  1.698494  4.302488   
3    0.1   1.0   0.1  1.211174e-03   8.447628  1.574803  4.350253   
4    0.1   1.0   1.0  9.926816e-04   7.488980  1.575495  4.351099   
5    0.1   1.0  10.0  9.620019e-04   5.699785  1.615442  4.326397   
6    0.1  10.0   0.1  8.723427e-04   4.751760  1.575203  4.350253   
7    0.1  10.0   1.0  4.801992e-04   4.636697  1.574880  4.350248   
8    0.1  10.0  10.0  9.036149e-04   4.045476  1.603087  4.315084   
9    1.0   0.1   0.1  1.332268e-15   7.611300  1.555677  4.326486   
10   1.0   0.1   1.0  4.577808e-04   6.527704  1.556046  4.326855   
11   1.0   0.1  10.0  1.063768e-03   4.882125  1.557585  4.328470   
12   1.0   1.0   0.1  4.574247e-05   6.233425  1.574791  4.350253   
13   1.0   1.0   1.0  7.819888e-04   5.636498  1.575255  4.350890   
14   1.0   1.0  10.0  9.529066e-05   4.350323  1.574782  4.350323   
15   1.0  10.0   0.1  4.086574e-04   3.744420  1.574716  4.350253   
16   1.0  10.0   1.0  3.454985e-04   3.656385  1.574791  4.350253   
17   1.0  10.0  10.0  3.790971e-04   3.199140  1.574694  4.350478   
18  10.0   0.1   0.1  8.169633e-04   3.052813  1.483031  4.283233   
19  10.0   0.1   1.0  2.574233e-03   2.920517  1.496308  4.297981   
20  10.0   0.1  10.0  5.113312e-04   2.420607  1.495527  4.298098   
21  10.0   1.0   0.1  1.213835e-03   2.913524  1.500809  4.302916   
22  10.0   1.0   1.0  2.571436e-03   2.803985  1.493295  4.297281   
23  10.0   1.0  10.0  7.894140e-04   2.351205  1.502714  4.307743   
24  10.0  10.0   0.1  4.744980e-04   2.249044  1.515284  4.310527   
25  10.0  10.0   1.0  4.430116e-04   2.210031  1.514216  4.309160   
26  10.0  10.0  10.0  2.813574e-04   1.981430  1.516841  4.311826   

           ratio  
                  
0   2.983783e-05  
1   2.624874e-04  
2   8.504914e-05  
3   1.433744e-04  
4   1.325523e-04  
5   1.687786e-04  
6   1.835831e-04  
7   1.035649e-04  
8   2.233643e-04  
9   1.750381e-16  
10  7.012891e-05  
11  2.178905e-04  
12  7.338255e-06  
13  1.387366e-04  
14  2.190427e-05  
15  1.091377e-04  
16  9.449184e-05  
17  1.184997e-04  
18  2.676100e-04  
19  8.814306e-04  
20  2.112409e-04  
21  4.166210e-04  
22  9.170648e-04  
23  3.357487e-04  
24  2.109777e-04  
25  2.004549e-04  
26  1.419971e-04

In [3]:
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS
from plotly.subplots import make_subplots
import re

fig = make_subplots(1, 3, subplot_titles=[
    f'$r_4 = {r3}$' for r3 in [0.1, 1, 10]
], horizontal_spacing=0.05, vertical_spacing=0.1)

for i, (r3, d1) in enumerate(sens.groupby('r3')):
    z = d1.sort_values(['r2', 'r1']).ratio.to_numpy().reshape(3, 3)
    z = np.flipud(z)
    fig.add_trace(go.Heatmap(
        z=z*100,
        x=['0.1', '1', '10'],
        y=['10', '1', '0.1'],
        coloraxis='coloraxis',
        text=z*100,
        texttemplate='%{text:.2f}%',
        textfont={'size': 8}
    ), row=1, col=i+1)

fig.update_layout(
    coloraxis={'colorscale': 'viridis'},
    height=300,
)
fig.update_xaxes(title='$r_2$', row=1)
fig.update_yaxes(title='$r_3$', col=1)
fig.show()
fig.write_image('../figures/sensitivity_mad_optim.png')